In [1]:
import numpy as np

#### Constants

In [2]:
SEED = 57
SAMPLING_FREQUENCY = 173.6

#### Set random seed into fixed value

In [3]:
import os
import random

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

#### Load signals

In [4]:
import pickle

x = pickle.load(open('../data/x.pkl', 'rb'))
y = pickle.load(open('../data/y.pkl', 'rb'))

In [5]:
x_normal = np.concatenate((x[:300], x[400:]), axis=0)
x_seizure = x[300:400]

print(x_normal.shape)
print(x_seizure.shape)

(400, 4097)
(100, 4097)


In [6]:
from scipy.signal import butter, lfilter

b, a = butter(3, [0.5,40], btype='bandpass',fs=SAMPLING_FREQUENCY)

x_normal_filtered = np.array([lfilter(b,a,x_normal[index,:]) for index in range(x_normal.shape[0])])
x_seizure_filtered = np.array([lfilter(b,a,x_seizure[index,:]) for index in range(x_seizure.shape[0])])

print(x_normal.shape)
print(x_seizure.shape)

x_normal = x_normal_filtered
x_seizure = x_seizure_filtered

(400, 4097)
(100, 4097)


#### Extract statistics features from dataset such as mean, std, var, min, max, etc.

In [7]:
def mean(data):
    return np.mean(data, axis=-1)

def std(data):
    return np.std(data, axis=-1)

def ptp(data):
    return np.ptp(data, axis=-1)

def var(data):
    return np.var(data, axis=-1)

def minimum(data):
    return np.min(data, axis=-1)

def maximum(data):
    return np.max(data, axis=-1)

def arg_min(data):
    return np.argmin(data, axis=-1)

def arg_max(data):
    return np.argmax(data, axis= -1)

def sqrt(data):
    return np.sqrt(np.mean(data**2, axis=-1))

def abs_diff_signal(data):
    return np.sum(np.abs(np.diff(data, axis=-1)), axis=-1)

def concat_statistics_features(data):
    return (mean(data), var(data), std(data), minimum(data), maximum(data))

In [8]:
statistics_features = []
for data in x_normal:
    statistics_features.append(concat_statistics_features(data))

statistics_features_array = np.array(statistics_features)
statistics_features_array.shape

(400, 5)

In [9]:
pip install antropy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Extract entropy features from dataset such as permutation, singular value decomposition, ...

In [32]:
import antropy as ent

# Permutation entropy
def permutation(data):
    return ent.perm_entropy(data, normalize=True)

# Spectral entropy
def spectral(data):
    return ent.spectral_entropy(data, sf=100, method='welch', normalize=True)

# Singular value decomposition entropy
def singular_value_decomposition(data):
    return ent.svd_entropy(data, normalize=True)

# Approximate entropy
def approximate(data):
    return ent.app_entropy(data)

# Sample entropy
def sample(data):
    return ent.app_entropy(x)

# Hjorth mobility and complexity
def hjorth_mobility(data):
    return ent.hjorth_params(x)

# Number of zero-crossings
def number_of_zerocross(data):
    return ent.num_zerocross(x)

# Lempel-Ziv complexity
def lempel_ziv(data):
    return ent.lziv_complexity('01111000011001', normalize=True)

def concat_entropy_features(data):
    return (permutation(data), spectral(data), singular_value_decomposition(data), approximate(data), lempel_ziv(data))

In [37]:
entropy_features = []
for data in x_normal:
    entropy_features.append(concat_entropy_features(data))

entropy_features_array = np.array(entropy_features)
entropy_features_array.shape

(400, 5)

In [43]:
def ptp(data):
    return np.ptp(data, axis=-1)

def twoop(data):
    return np.argmax(data) - np.argmin(data)

def pps(data):
    return ptp(data) / twoop(data)

print(ptp(x_normal[0]))
print(twoop(x_normal[0]))
print(pps(x_normal[0]))

167.39828589112523
3130
0.05348188047639783
